In [1]:
"""
imports
"""
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import math


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
"""
Variables
"""
#Down sampling voxel size
voxelSize = 0.5

#Ground filtering
z = 35

#Clustering by DBSCAN
eps = 1.5
min_points = 20

# RANSAC parameters
distance_threshold = 0.5
num_iterations = 1000
min_plane_ratio = 0.30

In [3]:
"""
Downsample point cloud using voxel downsampling
"""
data15pcd = o3d.io.read_point_cloud("Data15.pcd")
print("Original point count:", len(data15pcd.points))

data15pcdVoxelDownSample = data15pcd.voxel_down_sample(voxelSize)

print("Downsampled point count:", len(data15pcdVoxelDownSample.points))
reduction = (1 - len(data15pcdVoxelDownSample.points)/len(data15pcd.points)) * 100
print(f"Point cloud size reduced by {reduction:.2f}%")


Original point count: 9265401
Downsampled point count: 5306785
Point cloud size reduced by 42.72%


In [4]:
"""
Ground filtering
"""
points = np.asarray(data15pcdVoxelDownSample.points)
bbox = o3d.geometry.AxisAlignedBoundingBox(
    min_bound=(-np.inf, -np.inf, points[:,2].min()+z),
    max_bound=(np.inf, np.inf, points[:,2].max())
)
data15GroundFiltered = data15pcdVoxelDownSample.crop(bbox)
print("Filtered point count:", len(data15GroundFiltered.points))

o3d.io.write_point_cloud("Data15GroundFiltered.pcd", data15GroundFiltered)


Filtered point count: 1001586


True

In [5]:
"""
Clustering by DBSCAN
"""
data15GroundFiltered = o3d.io.read_point_cloud("Data15GroundFiltered.pcd")

labels = np.array(data15GroundFiltered.cluster_dbscan(
    eps=eps,
    min_points=min_points,
    print_progress=True
))
np.save("dbscanLabels.npy", labels)


numberOfClusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Number of clusters: {numberOfClusters}")

"""
Visualize DBSCAN clusters
"""
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = [0, 0, 0, 1]
data15GroundFiltered.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([data15GroundFiltered], window_name="DBSCAN Clusters")


Number of clusters: 2182
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [6]:
"""
RANSAC
"""

data15GroundFiltered = o3d.io.read_point_cloud("Data15GroundFiltered.pcd")
labels = np.load("dbscanLabels.npy")

uniqueLabels = np.unique(labels)
print(f"Running RANSAC on {len(uniqueLabels) - 1} clusters")

allRoofs = []


# RANSAC plane fitting per cluster 
for cluster_id in uniqueLabels:
    if cluster_id == -1:
        continue  # skip noise

    cluster = data15GroundFiltered.select_by_index(np.where(labels == cluster_id)[0])
    
    # First plane
    plane1, inliers1 = cluster.segment_plane(
        distance_threshold=distance_threshold, 
        ransac_n=3, 
        num_iterations=num_iterations
    )
    inlier1 = cluster.select_by_index(inliers1)
    outliers = cluster.select_by_index(inliers1, invert=True)

    if len(outliers.points) < 100:
        continue

    # Second plane
    plane2, inliers2 = outliers.segment_plane(
        distance_threshold=distance_threshold,
        ransac_n=3,
        num_iterations=num_iterations
    )
    inlier2 = outliers.select_by_index(inliers2)

    # Compute coverage ratios 
    ratio1 = len(inlier1.points) / len(cluster.points)
    ratio2 = len(inlier2.points) / len(cluster.points)

    if ratio1 < min_plane_ratio or ratio2 < min_plane_ratio:
        continue

    # Check angle between planes 
    n1 = np.array(plane1[:3])
    n2 = np.array(plane2[:3])
    angle = np.degrees(np.arccos(np.clip(np.dot(n1, n2) / (np.linalg.norm(n1) * np.linalg.norm(n2)), -1, 1)))

    if angle < 10:
        continue

    # Color and store valid roof planes 
    inlier1.paint_uniform_color([0, 0, 1])  
    inlier2.paint_uniform_color([0, 1, 0])
    allRoofs.extend([inlier1, inlier2])

print(f"Found {len(allRoofs)} valid roof planes total.")

# Visualization and export 
if allRoofs:
    mergedRoofs = o3d.geometry.PointCloud()
    for p in allRoofs:
        mergedRoofs += p

    o3d.visualization.draw_geometries([mergedRoofs], window_name="Detected Roof Planes")
    o3d.io.write_point_cloud("RANSACRoofsMerged.pcd", mergedRoofs)
else:
    print("No valid planes found.")


Running RANSAC on 2182 clusters
Found 266 valid roof planes total.
